## Plan
1. Charger les données
2. Split dataset
3. Pretraitement des données
       3.1. One hot encoding pour les données catégorielle
       3.2. Normalisation de données numériques (pour le Regression Logistique)
4. Random Forest
       4.1. Faire apprendre le modèle (par défault, RandomizedSearch, GridSearchCV, cross-validation, best_clf)
       4.2. Améliorer le modèle (parametrage, jouer avec le volume de données)
       4.3. Faire apprendre le modèle après le paramètrage de l'étape précédente
5. Feature importance
6. Conclusion

### 1. Charger les données

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("data/features_target.csv", sep=";")
df.head()

,id,slope,cd_water,aspect,hill90,hill5,hill45,clc00,numpoints,target
0,1,4.98010,0.256047,9,254,0,195,312,0,0
1,2,3.36810,0.296268,2,254,0,187,312,0,0
2,3,3.86523,0.374666,2,254,0,186,312,0,0
3,4,5.56061,0.471633,2,253,0,191,312,0,0
4,5,9.50320,0.589352,10,251,0,204,312,0,0


In [4]:
cat_cols = ["aspect", "clc00"]
num_cols = ["slope", "cd_water", "hill5", "hill45", "hill90"]

### 2. Split dataset

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
X = df.drop(["id", "numpoints", "target"], axis=1)
y = df.target

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.30)
X_train.shape, X_test.shape

((24678, 7), (10577, 7))

In order to not make the data leaks, we going to make all the data ingeneering after spliting the dataset

### 3. Pretraitement des données
#### 3.1. One hot encoding pour les données catégorielle

In [9]:
# encode categorical features in train set
X_train = pd.concat([X_train[num_cols],
                    pd.get_dummies(X_train.aspect),
                    pd.get_dummies(X_train.clc00)], axis=1)

# check if all is ok after data spliting
X_train.shape

(24678, 47)

In [10]:
# encode categorical features in test set
X_test = pd.concat([X_test[num_cols],
                    pd.get_dummies(X_test.aspect),
                    pd.get_dummies(X_test.clc00)], axis=1)

X_test.shape

(10577, 47)

In [11]:
# ici nous allons nous assurer que les deux jeux de données ont la même nombre de champs
print('Nombre de champs dans le train set:', len(X_train.columns))
print('Nombre de champs dans le test set: ', len(X_test.columns))

print(set(X_train.columns) - set(X_test.columns))
print(set(X_test.columns) - set(X_train.columns))

Количество признаков в train: 47
Количество признаков в valid: 47
{411}
{421}


In [14]:
# don't do that
X_train.drop([411], axis=1)
X_test.drop([421], axis=1)

,slope,cd_water,hill5,hill45,hill90,1,2,3,4,5,...,322,323,324,331,332,333,334,512,521,523
11431,12.208700,0.337930,0,145,249,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
26454,3.785660,0.084686,29,185,254,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
32394,6.963620,0.137171,28,183,253,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
14239,1.504800,0.158528,22,180,254,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4145,1.576570,0.000000,0,175,254,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28818,1.459390,0.132887,27,183,254,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
17265,1.564830,0.073344,18,177,254,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31724,7.530700,0.000000,0,175,252,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
12820,6.790660,0.113246,43,194,253,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


Maintenant nous avons 2 jeux de données
- train (19830 objects (rows))
- valid (6611 objects)
- test (8814 objects)

## Random Forest

### Faire apprendre le modèle
- avec les paramètres par défault, 
- RandomizedSearch, 
- GridSearchCV, 
- cross-validation, 
- best_clf

In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import roc_auc_score

Créer un object qui est un classificateur avec les parmètres par défault.

Nous allons fixer le paramètre random_state pour pouvoir réproduire les données par la suite

In [168]:
clf_rf = RandomForestClassifier(random_state=42)

clf_rf1 = clf_rf.fit(X_train, y_train)

La metrique qualité sera le ROC-AUC parce que notre class visé est unbalancé

In [169]:
# predictions
print('ROC-AUC train set:', roc_auc_score(y_train, clf_rf1.predict_proba(X_train)[:,1]))
print('ROC-AUC test set: ',  roc_auc_score(y_test, clf_rf1.predict_proba(X_test)[:,1]))

ROC-AUC для обучающей выборки:     0.9819728002890791
ROC-AUC для валидационной выборки: 0.6195884060868989


Метрика ROC-AUC показала высокое качество на обучающей выборке, но всего 62 качества на тестовой. На лицо явное переобучение модели. Постараемся улучшить её обучающую способность. Для начала воспользуемся RandomizedSearch, чтобы уменьшить диапазон рассматриваемых наилучших значений параметров модели. Затем воспользуемся GridSearchCV.

#### RandomizedSearch

In [ ]:
# RandomizedSearchCV

#### GridSearchCV

In [175]:
# GridSearchCV based on data obtained in RandomizedSearchCV

clf_rf = RandomForestClassifier(random_state=42)

params_rf = {'max_depth': [20, 26],
             'min_samples_split': [6, 7],
             'min_samples_leaf': [3, 5],
             'n_estimators':[200, 350, 600]}

search_rf_grid = GridSearchCV(clf_rf, params_rf, 
                                cv=5, n_jobs=-1)

search_rf_grid.fit(X_train, y_train)
search_rf_grid.best_params_

{'max_depth': 20,
 'min_samples_leaf': 3,
 'min_samples_split': 6,
 'n_estimators': 200}

In [176]:
# predictions
print('ROC-AUC train set:    ', roc_auc_score(y_train, search_rf_grid.predict_proba(X_train)[:,1]))
print('ROC-AUC test set :', roc_auc_score(y_test, search_rf_grid.predict_proba(X_test)[:,1]))

ROC-AUC для обучающей выборки:     0.9751290664807163
ROC-AUC для валидационной выборки: 0.616873292287433


Итак наилучшие параметры, полученные благодаря GridSearchCV sont

In [178]:
best_rf = RandomForestClassifier(n_estimators=800,
                                 criterion='entropy',
                                 max_depth=5,
                                 min_samples_split=4,  
                                 min_samples_leaf=1, 
                                 random_state=42)

best_rf.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', max_depth=5, min_samples_split=4,
                       n_estimators=800, random_state=42)

In [179]:
# predictions
y_pred = best_rf.predict_proba(X_test)[:,1]
# y_true is y_test

# score
print('ROC-AUC для обучающей выборки:    ', roc_auc_score(y_train, best_rf.predict_proba(X_train)[:,1]))
# print('ROC-AUC для валидационной выборки:',  roc_auc_score(y_valid, y_pred))
print('ROC-AUC для тестовой выборки:     ',  roc_auc_score(y_test, best_rf.predict_proba(X_test)[:,1]))

ROC-AUC для обучающей выборки:     0.7166262637876225
ROC-AUC для тестовой выборки:      0.6263397676407525


### 4.2. Améliorer le modèle (parametrage, jouer avec le volume de données)

Обучающая способность модели все еще неудовлетворительная. Чтобы понять, как улучшить модель, обратимся к литературе и веб источникам.

https://towardsdatascience.com/improving-random-forest-in-python-part-1-893916666cd

Есть три общих подхода к улучшению существующей модели машинного обучения:

- Используйте больше (высококачественных) данных и функций
- Настроить гиперпараметры алгоритма
- Попробуйте другие алгоритмы

Они представлены в том порядке, в котором я их обычно пробую.

- отбор фичей, уменьшение их количества
- Другие способы уменьшения количества признаков: PCA, ICA 

#### 4.2.1. Попробуем вручную подобрать параметры обучения модели.
Для начала посмотрим, что дает варьирование числа фолдов

In [ ]:
e = []
for n in range(3, 7):
    model_rf = RandomForestClassifier(random_state=42)
    params_grid = {'max_depth':[5], "n_estimators":[350], 'criterion':['entropy']}
    search_grid = GridSearchCV(model_rf, params_grid, n_jobs=-1, cv=n)
    search_grid.fit(X_train, y_train)
    y_pred = search_grid.predict_proba(X_train)[:,1]
    auc = roc_auc_score(y_train, y_pred)
    print("cv =", n, ":", auc)
    e.append(auc)

Итак, видим, что...

Возможно дело в недостатке данных? Проверим и это предположение

#### 4.2.2. Зависимость качества обучения от объема обучающей выборки
Проанализируем, как зависит качество модели от количества обучающих объектов выборки. 

источник: coursera - Spetialisation Data Science - Machine Learning

Для этого создадим модель. Для начала будем строить случайный лес над 50 деревьями, каждый из которых будет иметь глубину не больше 2

In [16]:
from sklearn.model_selection import cross_val_score, learning_curve
from sklearn import ensemble, metrics
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
clf_rf = ensemble.RandomForestClassifier(n_estimators = 50, max_depth = 2, random_state = 1)

И теперь давайте построим следующий график — нам будет интересно посмотреть, как меняется качество на обучающей и тестовой выборке, в зависимости от того, на скольких объектах мы обучаемся.

Для того чтобы получить такие графики, у sklearn есть специальная функция под названием learning_curve. Она позволяет нам сделать следующее — ей можно передать на вход нужный нам алгоритм, передать данные и целевую функцию, а также сказать, в каких пропорциях мы хотим обучаться, то есть на каких долях обучающей выборки мы хотим строить модель.

После этого с помощью этого метода будут построены несколько моделей, мы получим оценку качества на каждом объеме обучающей выборки, и нам будут возвращены размер обучающей выборки, оценки качества на «трейне» и оценка качества на тесте.

Имея такие данные, мы легко сможем проанализировать, как качество на обучении и тесте меняется от объема обучающей выборки. Вот давайте сделаем такую вещь.

Мы передаем в функцию наш классификатор, который мы создали ранее. Далее передаем туда данные, которые мы также подготовили на предварительном шаге. И говорим, что мы будем обучать модель на следующих данных: сначала мы возьмем 0,1 от обучающей выборки и далее будем двигаться с шагом 0,2 до 1. Оценивать качество будем с помощью уже знакомой нам метрики roc-auc и будем делать кросс-валидацию на 3 фолда. Давайте запустим.

In [ ]:
# max_depth=5, 100% of data

train_sizes, train_scores, test_scores = learning_curve(clf_rf, X_train, y_train, 
                                                        train_sizes=np.arange(0.1,1, 0.2), 
                                                        cv=5, scoring='roc_auc')

In [ ]:
print(train_sizes)
print(train_scores.mean(axis = 1))
print(test_scores.mean(axis = 1))

Для начала мы видим, что train_sizes — размер обучающей выборки — был преобразован из долей в конкретное количество объектов, на которых мы обучались. То есть мы видим, что минимальное количество обучающих объектов в рамках нашего эксперимента составляет 250, максимальное — 2250. Также нам доступны оценки качества на обучении и оценки качества на тесте. Так как у нас проводилась кросс-валидация, я сразу же сделала усреднение по всем фолдам — это делается с помощью команды mean. Аргумент axis = 1 означает, что мы будем усреднять по строчкам. Вот в данном случае каждая строка — это результат измерения кросс-валидации, поэтому нам это подходит.

Теперь давайте построим график. Сразу добавим на график сетку и будем строить две кривые — качество обучения на обучающей выборке и на тестовой выборке.

In [ ]:
pylab.grid(True)
pylab.plot(train_sizes, train_scores.mean(axis = 1), 'g-', marker='o', label='train')
pylab.plot(train_sizes, test_scores.mean(axis = 1), 'r-', marker='o', label='test')
pylab.ylim((0.0, 1.05))
pylab.legend(loc='lower right');

мы видим, что в начале качество на обучающей выборке падает — приблизительно до отметки 1250 деревьев, и дальше качество меняется очень медленно. С другой стороны, на тестовой выборке качество продолжает расти приблизительно до этой же точки, и дальше оно также перестает меняться. Какой вывод мы можем сделать из этого? Дальнейший рост обучающей выборки вряд ли скажется на качестве нашей модели. Это говорит о том, что модель данной сложности не может многое выиграть за счет того, что мы обогатим данные. Что же делать в такой ситуации? Кривые обучения для деревьев большей глубины

Давайте попробуем увеличить сложность модели — возможно, это приведет к улучшению ее качества. Так как мы с вами обучали модель на деревьях глубины 2, давайте увеличим глубину деревьев — это даст нам дополнительные возможности.

Снова создаем классификатор RandomForestClassifier, но в этот раз указываем ему параметр max_depth = 10 — это максимально возможная глубина деревьев.

In [ ]:
# max_depth=18
clf_rf = ensemble.RandomForestClassifier(n_estimators = 600, max_depth = 2, random_state = 1)


# Теперь давайте еще раз запустим команду learning_curve и 
# построим кривую обучения на тесте и на обучении, при этом 
# мы будем делать это по тем же самым точкам, по тем же 
# самым долям обучающей выборки.
train_sizes, train_scores, test_scores = learning_curve(clf_rf, X_train, y_train, 
                                                        train_sizes=np.arange(0.1,1, 0.2), 
                                                        cv=5, scoring='roc_auc')
print(train_sizes)
print(train_scores.mean(axis = 1))
print(test_scores.mean(axis = 1))

pylab.grid(True)
pylab.plot(train_sizes, train_scores.mean(axis = 1), 'g-', marker='o', label='train')
pylab.plot(train_sizes, test_scores.mean(axis = 1), 'r-', marker='o', label='test')
pylab.ylim((0.0, 1.05))
pylab.legend(loc='lower right');

#### Results

При 50 деревьях не хватает обобщающей способности модели

#### 4.2.3. Imputing the value to target class

(résoudre le problème de classes unbalancées)

Попробуем решить проблему несбалансированности выборки.

    a - imputing
    b - changer le datase, obtenit plus de données

### 4.2.4. Feature Importance

Отбор признаков является важным этапом построения алгоритмов машинногообучения. Данный этап необходим, чтобы избавиться от шумовых признаков и бла-годаря этому улучшить качество и ускорить работу алгоритмов. Проведенные экспе-рименты подтверждают, что алгоритмы отбора признаков с помощью Random Forestэффективно справляется со своей задачей
http://www.machinelearning.ru/wiki/images/9/96/RysmyatovaCourseFile.pdf

In order to quantify the usefulness of all the variables in the entire random forest, we can look at the relative importances of the variables. The importances returned in Skicit-learn represent how much including a particular variable improves the prediction. The actual calculation of the importance is beyond the scope of this post, but we can use the numbers to make relative comparisons between variables.

Крооме того, опеределение и классификация важности признаков привность некоторые интересные элементы в вопрос факторах, характеризующих места, наиболее пригодные для заселения древними. Характеристики территорий, на которых было найдено больше всего следов пребывания, стоянок.

In [ ]:
# Код для отрисовки важности фичей 1

imp = pd.DataFrame(best_rf.feature_importances_, index=X_train.columns, columns=['importance'])
imp.sort_values('importance').plot(kind='barh', figsize=(12, 8));